task https://stepik.org/lesson/701337/step/6?unit=701406

In [12]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://parsinger.ru/html/index1_page_1.html'
site = 'http://parsinger.ru/html/'

def get_response(url):
    response = requests.get(url=url)
    response.encoding = 'utf-8'  
    return response

def cook_pagination(response,site):
    soup = BeautifulSoup(response.text, 'lxml')
    pages = [f"{site}{link['href']}" for link in soup.find('div', class_='pagen').find_all('a')]        
    return pages

def flatten(list_of_lists):
    if len(list_of_lists) == 0:
        return list_of_lists
    if isinstance(list_of_lists[0], list):
        return flatten(list_of_lists[0]) + flatten(list_of_lists[1:])
    return list_of_lists[:1] + flatten(list_of_lists[1:])

def cook_products(url, response):
    soup = BeautifulSoup(response.text, 'lxml')
    categories = 'watch'
    name = soup.find("p", id='p_header' ).text
    article = soup.find('p', class_='article').text.split()[1]
    description = [x.text.split('\n') for x in soup.find('ul', id ='description')]
    description = flatten(description)
    description = [value for value in description if value]
    availability = soup.find("span", id='in_stock').text    
    price = soup.find('span', id='price').text
    old_price = soup.find('span', id='old_price').text
    
    flat = categories, name, article, *[x.split(':')[1].strip() for x in description if x], availability.split()[2], price, old_price, url
    return flat

def sbor_imen(url, response):
    soup = BeautifulSoup(response.text, 'lxml')
    description = soup.find('ul', id='description').find_all('li')
    li_id = [x['id'] for x in description]
    return li_id    


for page in cook_pagination(get_response(url),site):
    response = get_response(page)
    soup = BeautifulSoup(response.text, 'lxml')   
    result_json = []
    url_products = [f"{site}{link['href']}" for link in soup.find_all('a', class_='name_item')]
    li_id = sbor_imen(url_products[0], get_response(url_products[0]))
    for url_product in url_products:
        list_item = cook_products(url_product, get_response(url_product))    
        
        result_json.append({
            'categories': [x.strip() for x in list_item][0],
            'name': [x.strip() for x in list_item][1],
            'article': [x.strip() for x in list_item][2],
            'description':{
            f'{li_id[0]}': [x.strip() for x in list_item][3],
            f'{li_id[1]}': [x.strip() for x in list_item][4],
            f'{li_id[2]}': [x.strip() for x in list_item][5],
            f'{li_id[3]}': [x.strip() for x in list_item][6],
            f'{li_id[4]}': [x.strip() for x in list_item][7],
            f'{li_id[5]}': [x.strip() for x in list_item][8],
            f'{li_id[6]}': [x.strip() for x in list_item][9],
            f'{li_id[7]}': [x.strip() for x in list_item][10]
            },
            'count': [x.strip() for x in list_item][11],
            'price': [x.strip() for x in list_item][12],
            'old_price': [x.strip() for x in list_item][13],
            'link': [x.strip() for x in list_item][14]
            })
        
    with open('res3.json', 'a', encoding='utf-8') as file:
        json.dump(result_json, file, indent=4, ensure_ascii=False)

print('Файл res3.json создан')



Файл res3.json создан
